# Workshop 3: Data Pre-processing¶
COSC2671 Social Media and Network Analytics

Jeffrey Chan, RMIT University, 2023


The following function is a helper one, it does tokenisation removing stopwords and stemming.

In [1]:
def processText(text, tokenizer, stemmer, stopwords):
    """
    Perform tokenisation, normalisation (lower case and stemming) and stopword and twitter keyword removal.

    @param text: reddit submission or comment text
    @param tokenizer: tokeniser used.
    @param stemmer: stemmer used.
    @param stopwords: list of stopwords used

    @returns: a list of processed tokens
    """

    # covert all to lower case
    text = text.lower()
    # tokenise
    lTokens = tokenizer.tokenize(text)
    # strip whitespaces before and after
    lTokens = [token.strip() for token in lTokens]
    # stem (we use set to remove duplicates)
    lStemmedTokens = set([stemmer.stem(tok) for tok in lTokens])


    # remove stopwords, digits
    return [tok for tok in lStemmedTokens if tok not in stopwords and not tok.isdigit()]

The following is the main part of the Notebook.  It setups, loads from the json file and calls processText().  Then it update the count of the terms.

In [2]:
import sys
import json
import string
from collections import Counter
import nltk
nltk.download('stopwords')
import matplotlib.pyplot as mpl


# load json file
# note usually we would do some checks, but for clarify's sake we haven't implement that code here
fJsonName = 'australiaSubreddit.json'

# number of most frequent terms to display
freqNum = 50

# tweet tokeniser to use
tweetTokeniser = nltk.tokenize.TweetTokenizer()
# use the punctuation symbols defined in string.punctuation
lPunct = list(string.punctuation)
# use stopwords from nltk and a few other twitter specific terms like 'rt' (retweet)
lStopwords = nltk.corpus.stopwords.words('english') + lPunct + ['via']
# we use the popular Porter stemmer
tweetStemmer = nltk.stem.PorterStemmer()

# our term frequency counter
termFreqCounter = Counter()



# open json file and process it tweet by tweet
with open(fJsonName, 'r') as f:
    dSubmissions = json.load(f)
    
    for submission in dSubmissions['submissions']:
        submissionsTitle = submission.get('title', '')
        # tokenise, filter stopwords and get convert to lower case
        lTokens = processText(text=submissionsTitle, tokenizer=tweetTokeniser, stemmer=tweetStemmer, stopwords=lStopwords)

        # update count
        termFreqCounter.update(lTokens)

# print out most common terms
for term, count in termFreqCounter.most_common(freqNum):
    print(term + ': ' + str(count))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


’: 8
australia: 7
australian: 7
thi: 5
anyon: 4
‘: 3
whi: 3
jobseek: 3
still: 3
increas: 3
ani: 3
movi: 2
miss: 2
cost: 2
price: 2
live: 2
find: 2
scott: 2
morrison: 2
care: 2
doe: 2
logi: 2
wait: 2
move: 2
—: 2
travel: 2
forc: 2
indigen: 2
data: 2
nsw: 2
report: 2
alreadi: 2
govern: 2
accus: 2
rim: 2
scenic: 2
owner: 2
dog: 2
stupid: 2
threat: 2
fortnight: 2
beach: 2
onc: 2
week: 2
ga: 2
militari: 2
us: 2
pay: 2
last: 2
payment: 2
